
# 1. Preparação



In [ ]:
!pip install ultralytics
# !pip install roboflow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 35.7 MB/s eta 0:00:00


In [ ]:
# 1: Montar Google Drive e Instalar Bibliotecas
from IPython import display

# display.clear_output()

from google.colab import drive
import os
import glob
import cv2
import numpy as np
import pandas as pd
from ultralytics import YOLO

# ultralytics.checks()

from IPython.display import display, Image
import torch
from torchvision import models, transforms
from PIL import Image
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

drive.mount('/content/drive')
# Certifique-se de que todas as bibliotecas necessárias estejam instaladas, se faltar alguma!

# Definições de Caminho (AJUSTE CONFORME SEU PROJETO)
DRIVE_PROJECT_PATH = '/content/drive/MyDrive/Colab Notebooks/RobotPerception/LookStock'
IMAGES_INPUT_DIR = f'{DRIVE_PROJECT_PATH}/dataset' # <--- ONDE ESTÃO AS 412 IMAGENS
OUTPUT_DIR = f'{DRIVE_PROJECT_PATH}/meus_resultados'

MODEL_YOLO_PATH = f'{DRIVE_PROJECT_PATH}/models/best.pt'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Garante que os diretórios de saída existam
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'imagens_processadas'), exist_ok=True)
os.makedirs(os.path.join(OUTPUT_DIR, 'relatorios_individuais'), exist_ok=True)

print(f"Dispositivo de processamento: {DEVICE}")

# os.chdir('/content/drive/MyDrive/Colab Notebooks/RobotPerception/LookStock')
# !ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Dispositivo de processamento: cpu


# 2. Funções de Identificação e Análise

In [ ]:
# 2: Base de Conhecimento e Planograma
# ----------------------------------------------------------------------
# ⚠️ Substitua pelos seus embeddings de referência reais
KNOWLEDGE_BASE = {
  'Wonder_White': np.random.rand(2048),
  'Sunblest_White': np.random.rand(2048),
  'Minky_Soft': np.random.rand(2048),
  'Countdown_White': np.random.rand(2048)
}

# Célula 2: Base de Conhecimento e Embeddings

# ⚠️ SUBSTITUA PELOS SEUS DADOS REAIS
# KNOWLEDGE_BASE = {
#     'CocaCola_Lata': np.random.rand(2048),
#     'Fanta_Laranja': np.random.rand(2048),
#     'Sprite_Pet': np.random.rand(2048),
# }

PLANOGRAMA_IDEAL = {
  'Wonder_White': 10,
  'Sunblest_White': 8,
  'Minky_Soft': 5,
  'Countdown_White': 3
}
SIMILARITY_THRESHOLD = 0.70 # Limite para considerar um SKU identificado

# Inicialização do Modelo de Embeddings (ResNet50)
embedding_model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
embedding_model = torch.nn.Sequential(*(list(embedding_model.children())[:-1]))
embedding_model.to(DEVICE)
embedding_model.eval()

preprocess = transforms.Compose([
    transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

def get_embedding(image_crop_np):
    """Gera o vetor de embedding na GPU."""
    try:
        img_pil = Image.fromarray(cv2.cvtColor(image_crop_np, cv2.COLOR_BGR2RGB))
        img_tensor = preprocess(img_pil).unsqueeze(0).to(DEVICE)
        with torch.no_grad():
            embedding = embedding_model(img_tensor).cpu().numpy().flatten()
        return embedding
    except:
        return None

def identify_product(new_embedding, knowledge_base=KNOWLEDGE_BASE, similarity_threshold=SIMILARITY_THRESHOLD):
    """Compara o embedding e retorna o SKU mais provável."""
    if new_embedding is None: return "SKU_DESCONHECIDO"

    max_similarity = -1
    best_match_sku = "SKU_DESCONHECIDO"
    new_embedding = new_embedding.reshape(1, -1)

    for sku, ref_embedding in knowledge_base.items():
        ref_embedding = ref_embedding.reshape(1, -1)
        similarity = cosine_similarity(new_embedding, ref_embedding)[0][0]

        if similarity > max_similarity:
            max_similarity = similarity
            best_match_sku = sku

    if max_similarity > similarity_threshold:
        return f"{best_match_sku}"
    else:
        return f"SKU_DESCONHECIDO"

# 3. Pipeline de Processamento em Lote

In [ ]:
# Célula 3: Funções de Processamento em Lote

def process_shelf_image(image_path, yolo_model):
    """Processa uma única imagem: detecção, identificação e anotação visual de Rupturas."""

    img_bgr = cv2.imread(image_path)
    if img_bgr is None:
        return None, None, {"status": "Erro: Imagem não encontrada ou inválida"}

    img_rgb = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2RGB)
    contagem_real = {}
    total_empty_shelves = 0
    summary_data = {'image_file': os.path.basename(image_path)}

    # 1. Executar YOLO
    results = yolo_model.predict(source=image_path, save=False, conf=0.5, device=DEVICE.type, verbose=False)

    if not results or not results[0].boxes:
        summary_data['ruptura_vazio_direto'] = 0
        summary_data['ruptura_planograma_unid'] = 0
        summary_data['status_geral'] = 'Nenhuma Detecção'
        return img_rgb, contagem_real, summary_data

    names = results[0].names
    detected_boxes = results[0].boxes.xyxy.cpu().numpy()
    detected_classes = results[0].boxes.cls.cpu().numpy()
    detected_confidences = results[0].boxes.conf.cpu().numpy()

    # 2. Percorrer cada detecção e Anotar
    for box, cls_id, conf in zip(detected_boxes, detected_classes, detected_confidences):
        x1, y1, x2, y2 = map(int, box)
        class_name = names[int(cls_id)]

        # Lógica de Visualização e Ruptura (Baseado no ShelfSight)
        if class_name == 'emptySpaces' or class_name == 'empty_shelf': # 🔴 CLASSE DE RUPTURA
            # Cor Vermelha Escura para RUPTURA
            color_rgb_text = (178, 34, 34)
            label = f"RUPTURA {conf:.2f}"
            total_empty_shelves += 1

        else:
            # Produto: Tenta identificar via Embeddings
            product_crop = img_bgr[y1:y2, x1:x2]
            product_id = identify_product(get_embedding(product_crop))
            contagem_real[product_id] = contagem_real.get(product_id, 0) + 1

            if product_id == "SKU_DESCONHECIDO":
                # Produto não identificado (Laranja para revisão)
                color_rgb_text = (255, 165, 0)
                label = f"SKU_DESC {conf:.2f}"
            else:
                # Produto identificado (Verde Carregado)
                color_rgb_text = (0, 128, 0)
                label = f"{product_id} {conf:.2f}"

        # Anotar a imagem
        cv2.rectangle(img_rgb, (x1, y1), (x2, y2), color_rgb_text, 2)
        cv2.putText(img_rgb, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color_rgb_text, 2)

    # 3. Análise de Ruptura por Planograma (Complementar à detecção direta)
    rupturas_por_planograma_unid = 0
    for sku_esperado, ideal_count in PLANOGRAMA_IDEAL.items():
        real_count = contagem_real.get(sku_esperado, 0)
        if real_count < ideal_count:
            rupturas_por_planograma_unid += (ideal_count - real_count)

    # 4. Construção do Resumo
    summary_data['ruptura_vazio_direto'] = total_empty_shelves
    summary_data['ruptura_planograma_unid'] = rupturas_por_planograma_unid
    summary_data['status_geral'] = 'Alerta' if (total_empty_shelves > 0 or rupturas_por_planograma_unid > 0) else 'OK'

    return img_rgb, contagem_real, summary_data


def process_all_images(input_dir, output_dir, model_path):
    """Função principal para processar todas as imagens em lote."""
    print("Iniciando processamento em lote...")

    yolo_model = YOLO(model_path)
    image_files = glob.glob(os.path.join(input_dir, '*.*'))
    if not image_files:
        print(f"ERRO: Nenhuma imagem encontrada no diretório: {input_dir}")
        return

    all_summaries = []

    for i, img_path in enumerate(image_files):
        print(f"({i+1}/{len(image_files)}) Processando: {os.path.basename(img_path)}...")
        base_name = os.path.splitext(os.path.basename(img_path))[0]

        processed_img, real_counts, summary = process_shelf_image(img_path, yolo_model)

        if processed_img is None:
            summary['status_geral'] = 'ERRO DE ARQUIVO'
            all_summaries.append(summary)
            continue

        # Adiciona a contagem real ao resumo
        for sku_name in PLANOGRAMA_IDEAL.keys():
            summary[f'Contagem_{sku_name}'] = real_counts.get(sku_name, 0)

        all_summaries.append(summary)

        # Salvar Imagem Processada
        output_image_path = os.path.join(output_dir, 'imagens_processadas', f'{base_name}_processed.jpg')
        cv2.imwrite(output_image_path, cv2.cvtColor(processed_img, cv2.COLOR_RGB2BGR))

    # Gerar Relatório CSV Final
    df_summary = pd.DataFrame(all_summaries)
    for sku_name in PLANOGRAMA_IDEAL.keys():
        if f'Contagem_{sku_name}' not in df_summary.columns:
            df_summary[f'Contagem_{sku_name}'] = 0
    df_summary = df_summary.fillna(0)

    final_csv_path = os.path.join(output_dir, 'RESUMO_GLOBAL_RUPTURAS.csv')
    df_summary.to_csv(final_csv_path, index=False)

    print("\n" + "="*50)
    print("✅ Processamento em Lote CONCLUÍDO!")
    print(f"Resumo global salvo em: {final_csv_path}")
    print("="*50)

In [ ]:
# Célula 4: EXECUÇÃO
# ⚠️ GARANTA QUE O MODELO YOLO FOI TREINADO COM A CLASSE 'emptySpaces' ⚠️
process_all_images(IMAGES_INPUT_DIR, OUTPUT_DIR, MODEL_YOLO_PATH)

Iniciando processamento em lote...
(1/18) Processando: 0c2cc2c7-243a-4194-821b-ec0682cfe42e_jpg.rf.358957deb4b032d1b6b99f586ab0000b.jpg...
(2/18) Processando: 0e09c4a1-cbee-4c54-9b63-cf7e15ba790c_jpg.rf.58970a830cc69a3ad8c8a047fb02a930.jpg...
(3/18) Processando: 00eb59e1-ee3d-4ea4-81be-2698eeb9018d_jpg.rf.fd8a6928713d0f71587ccb576fdba46a.jpg...
(4/18) Processando: 25b84ba7-ac2d-4bef-bb47-b75e44a7f3e0_jpg.rf.79f9bf404ac6a1e13009639ee545047b.jpg...
(5/18) Processando: 1a5a8f13-80bf-404c-915a-ec5bd52c3453_jpg.rf.acc4297a6c599a316d50ca95019be039.jpg...
(6/18) Processando: 25e6a0f5-a5e0-445e-becf-59e4d3c3e438_jpg.rf.4e47f4870e407e411e63f5b629d23062.jpg...
(7/18) Processando: 1ea3cd9f-6b24-46c6-906d-7b97651c276f_jpg.rf.0b1c35b682684a6293422ccc6fc2af89.jpg...
(8/18) Processando: 58c5cd28-70ad-4e7b-a458-cd0d6400f53f_jpg.rf.713f334af9000a2bca4a17ca288d8f2e.jpg...
(9/18) Processando: 58b6a885-7ba0-490d-9a3b-ea98bd99dc65_jpg.rf.020235590dd3698b92f0bd59f0739b14.jpg...
(10/18) Processando: 6d19a01c